In [22]:
import requests # http requests
import re # regular expressions
from bs4 import BeautifulSoup # xml parsing
import pandas as pd
import regex as re
import os
import time
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm

In [23]:
url = 'https://kbhkort.kk.dk/spatialmap?page=widget-view&name=motion/motionslisten'

In [83]:
data = {'type':[], 'website':[], 'location':[], 'address':[]}

options = ChromeOptions()       # Get Chrome options
options.headless = True         # This stops an actual browser from being open and shown

driver = Chrome(r"C:\Users\chris\OneDrive\MSc\KDS_A2023_Web\chromedriver-win64\chromedriver.exe", options=options)  # Optional argument, if not specified will search path.
driver.get(url)

print("Driver and URL passed. Wait a second...")
driver.implicitly_wait(20) # Let the user actually see something!

count = len(driver.find_elements(By.XPATH, '/html/body/div/div[4]/div/ul/li'))
print(f'There is a total of {count} entries.')

for i in tqdm(range(1, count+1)): # count+1
    # Type of activity
    type = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[1]/strong[1]')
    data['type'].append(type.text)

    # Location of activity - not all entries list a location
    try:
        location = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[1]/strong[2]')
        data['location'].append(location.text)
    except NoSuchElementException:
        data['location'].append(None)
    # Website
    try:
        site = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[2]/span[1]/a')
        data['website'].append(site.get_attribute('href'))
    except NoSuchElementException:
        data['website'].append(None)

    # Address of activity - some entries have an extra field, so this messess with the current (janky) XPath implementation
    try:
        address = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[2]/span[6]')
        if not address.text.startswith('Mødested: '): # Check that the text is indeed the address
            try:
                address = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[2]/span[7]')
                data['address'].append((address.text).removeprefix('Mødested: '))
            except NoSuchElementException:
                data['address'].append(None)
        else: 
            data['address'].append((address.text).removeprefix('Mødested: '))
    except NoSuchElementException:
        data['address'].append(None)

driver.quit()

Driver and URL passed. Wait a second...
There is a total of 610 entries.


100%|██████████| 610/610 [57:20<00:00,  5.64s/it]  


In [86]:
df = pd.DataFrame.from_dict(data)
print(df.head())


,type,website,location,address
605,Mix Volley,https://ifk98.dk/default.aspx,Idrætsforeningen Kæmperne,"Øster Farimagsgade Skole, Multisalen"
606,Motionsgymnastik,https://ifk98.dk/default.aspx,Idrætsforeningen Kæmperne,None
607,Cykeltur - roligt tempo,https://ifk98.dk/default.aspx,Idrætsforeningen Kæmperne,None
608,Varmtvandsgymnastik,https://ifk98.dk/default.aspx,Idrætsforeningen Kæmperne,None
609,Bordtennis,https://ifk98.dk/default.aspx,Idrætsforeningen Kæmperne,Remisen


In [85]:
cwd = os.getcwd()
path = cwd + "/kbh_exercise.csv"
df.to_csv(path, encoding='utf-16')